# **Imports**

In [71]:
from pypdf import PdfReader
import re
import os

# **Configs**

In [72]:
PDF_DIRECTORY = "./inputPDFs/"

# **Process all files**

Source: https://pypdf.readthedocs.io/en/stable/user/extract-text.html

In [73]:
pdfs = os.listdir(PDF_DIRECTORY)
for pdf in pdfs:
    # extract filename without extension
    party = pdf.split(".")[0]
    reader = PdfReader(PDF_DIRECTORY + pdf)
    pages = reader.pages
    raw_text = ""
    lineNumber = 0
    clearedTexts = []
    meltPart = ""
    for page in pages:
        text = page.extract_text()
        text = text.replace('\n', ' ')
        text = text.replace(' - ', '')
        text = text.replace(' -', '')
        text = text.replace('- ', '')
        # remove whitespaces
        text = " ".join(text.split())
        # remove gender stars
        text = text.replace('*', '')
        #*
        if pdf == "DIE_GRUENEN.pdf":
        # remove this string Bundestagswahlprogramm 2021 BÜNDNIS 90 / DIE GRÜNEN 258, the number can differ
            text = re.sub(r'Bundestagswahlprogramm \d+ BÜNDNIS 90 / DIE GRÜNEN \d+', '', text)
            raw_text += text        
    
        if pdf == "AFD.pdf":
            chapter_pattern = r'(.+KAPITEL \d+)'
            # Search for the chapter pattern "KAPITEL x" in the text
            match = re.search(chapter_pattern, text)
            
            if match:
                chapter_name = match.group(1)
                
                # Extract the chapter number as a separate string
                chapter_match = re.search(r'KAPITEL \d+', text)
                chapter_string = chapter_match.group(0)
                
                # Extract the chapter name without the chapter number
                chapter_name = chapter_name.replace(chapter_string, "")
        
            # Remove chapter headings from the text
            text = re.sub(rf'{chapter_name} \d+ ', '', text)
            text = re.sub(rf'\d+ {chapter_name}', '', text)
            text = re.sub(rf'\s*\d+{chapter_string}', '', text)
            text = re.sub(rf'{chapter_string}\s*\d+', '', text)
            text = text.replace(chapter_name, '')
            raw_text += text

        if pdf == "CDU-CSU.pdf":
            text = re.sub(r'Seite \d+ von \d+', '', text)
            text = text.split(" ")
            cleared_text = []
            indexes = []
            for word in text:
                if meltPart != "":
                    word = meltPart + word
                    word = word.replace("-", "")
                    meltPart = ""
                index = 0
                digits = [int(match) for match in re.findall(r'\d+', word)]
                # Extract all digits from the word
                # Check if digit is equal to the line number
                # if yes the digit equals the line number and has to be removed
                for digit in digits:
                    # Sonderlocke CO21477
                    if digit == 21477:
                        word = "CO2"
                        lineNumber += 1

                    if digit == lineNumber or digit == lineNumber+1:
                        lineNumber += 1
                        strDigit = f"{digit}"

                        # e.g. konventio1881 
                        if strDigit != word:
                            words = [element for element in text if strDigit in element][0]
                            indexes.append(text.index(words))
                            word = word.replace(strDigit, '')
                        word = word.replace(strDigit, '')
                cleared_text.append(word)

            indexDecrease = 0
            for index in indexes:
                index -= indexDecrease
                # len = 50 -> melt e.g index12 and 13 -> index 14 is now index 13
                if index != len(cleared_text)-1:
                    cleared_text[index] = cleared_text[index] + cleared_text[index+1]
                    del cleared_text[index+1]
                    indexDecrease += 1
                # Special case: end of page is reached and word continues on next page
                else:
                    meltPart = cleared_text[index]
                    del cleared_text[index]
            # convert into string and remove •
            cleared_text = " ".join(list(filter(None, cleared_text))).replace("•", "")
            raw_text += cleared_text


        if pdf == "SPD.pdf":
            text = text.replace("SPD-Parteivorstand 2021", '')
            text = text.replace("Das Zukunftsprogramm der SPD", '')
            text = re.sub(r'Kapitel \d+ Seite >\d+', '', text)
            text = re.sub(r'Kapitel \d+', '', text)
            # Replace multiple whitespaces with one whitespace
            text = re.sub(r'\s+', ' ', text)
            raw_text += text
        
        if pdf == "DIE_LINKE.pdf":
            # Find all words containing characters and numbers -> numbers are page numbers
            matches = re.findall(r'\w*\d\w*', text)
            for match in matches:
                if not match.isdigit():
                    # Remove the page number from the string like "7Einführung"
                    cleaned = re.sub(r'\d', ' ', match)
                    text = text.replace(match, cleaned)
            text = text.replace(" n ", "")
            raw_text += text

        
        if pdf == "FDP.pdf":
            pattern = r'Das Programm der Freien Demokraten zur Bundestagswahl 2021 (\d+)'
            page = re.findall(pattern, text)
            if page:
                text = text.replace(f"Das Programm der Freien Demokraten zur Bundestagswahl 2021 {page[0]}", '')
            raw_text += text

    # write text to file
    with open(f'./output/{party}.txt', 'w') as f:
        f.write(raw_text)
